In [0]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/puigalex/AMP-Tech.git

In [0]:
%cd /content/drive/My Drive/obj_clasi

In [0]:
!ls

Clasificación_de_comida.ipynb  clasificación_de_comida.py  data  evaluation


In [0]:
from google.colab import files

Entrenar

import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
K.clear_session()
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'
"""
Parametros
"""
epocas=20 #Numero de iteraciones sobre el set de datos durante el entrnamiento
longitud, altura = 150, 150 #tamaño de imagen para el entrenamiento
batch_size = 32 #numero de imagenes por paso
pasos = 1000 #numero de veces que se procesara la información en cada éppoca
validation_steps = 300 #Al final de las épocas se realizara una validacion de 300 pasos
filtrosConv1 = 32 #Numero de filtros aplicados por combolución (Profundidad)
filtrosConv2 = 64 #
tamano_filtro1 = (3, 3) #tamaño del filtro usado en cada combolución
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2) #tamaño del filtro de max pooling
clases = 3 #numero de clases
lr = 0.0004 #coeficiente de aprendizaje


##Preparamos nuestras imagenes
#preprocesamiento
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, #reescalar las imágenes rango de pixeles de 0 a 1
    shear_range=0.2, #Generar imagenes inclinadas
    zoom_range=0.2,  #posibilidad de hacer zoom
    horizontal_flip=True)#invertir imagen 
test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')
#Red neuronal combolucional
cnn = Sequential()#red secuencial
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu')) #creacion de la primera capa 
cnn.add(MaxPooling2D(pool_size=tamano_pool))#capa de pooling

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))#segunda caoa combolucional
cnn.add(MaxPooling2D(pool_size=tamano_pool))#segunda capa de pooling
#inicia la clasificación
cnn.add(Flatten())#transformar la información a una sola dimencion 
cnn.add(Dense(256, activation='relu'))#256 neuronas, con activación relu
cnn.add(Dropout(0.5))#Se apagaran el 50% de las neuronas para evitar sobreajuste, para adaptarse a informacion nueva
cnn.add(Dense(clases, activation='softmax'))#realiza la clasificación

cnn.compile(loss='categorical_crossentropy',#parametos que se utilizaran para el algoritmo
            optimizer=optimizers.Adam(lr=lr),#optimizador adam
            metrics=['accuracy'])#metrica utilizada

#entrenamiento
cnn.fit_generator(
    entrenamiento_generador,#imagenes de entrenamiento
    steps_per_epoch=pasos,#numero de pasos
    epochs=epocas,#numero de pasos
    validation_data=validacion_generador,#imagenes de validazion
    validation_steps=validation_steps)#pasos de validación
#guardar modelo como archivo
target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

predict

import numpy as np
import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import tensorflow as tf 
longitud, altura = 150, 150#tamaño de la imagen
modelo = './modelo/modelo.h5'#direccion del modelo
pesos_modelo = './modelo/pesos.h5'#direccion de los pesos
cnn = tf.keras.models.load_model(modelo)#cargar modelo
cnn.load_weights(pesos_modelo)#cargar pesos
#funcion de prediccion
def predict(file):
  x = tf.keras.preprocessing.image.load_img(file, target_size=(longitud, altura))#cargar la imagen
  x = tf.keras.preprocessing.image.img_to_array(x)#tansformar imagen a arreglo
  x = np.expand_dims(x, axis=0)#en el eje 0 se agrega una dimención extra para procesar la información sin problema
  array = cnn.predict(x)#se llama a la red para realizar la predicción
  result = array[0]#obtenemos el resultado
  answer = np.argmax(result)#nos entrega el indice del valor mas alto
  #clasificación del resultado
  if answer == 0:
    print("pred: Perro")
  elif answer == 1:
    print("pred: Gato")
  elif answer == 2:
    print("pred: Gorila")

  return answer

print("cat")
file='./test/cat.5000.jpg'
predict(file)#ingresar direccion de la imagen
print("dog")
file='./test/dog.5000.jpg'
predict(file)#ingresar direccion de la imagen
print("gorilla")
file='./test/8.jpeg'
predict(file)#ingresar direccion de la imagen

New

In [1]:
import sys
import os
import tensorflow as tf
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import applications

In [2]:
vgg=applications.vgg16.VGG16()

In [3]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
cnn=Sequential()
for capa in vgg.layers:
    cnn.add(capa)

In [5]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
cnn.pop()

In [7]:
for layer in cnn.layers:
    layer.trainable=False

In [8]:
cnn.add(Dense(3,activation='softmax'))

In [9]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [10]:
def modelo():
    vgg=applications.vgg16.VGG16()
    cnn=Sequential()
    for capa in vgg.layers:
            cnn.add(capa)
    cnn.layers.pop()
    for layer in cnn.layers:
            layer.trainable=False
    cnn.add(Dense(11,activation='softmax'))
    
    return cnn

In [14]:
K.clear_session()

data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'


epocas=100
longitud, altura = 224, 224
batch_size = 20
pasos = 100
validation_steps = 100
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 20
lr = 0.1


##Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

##CREAR LA RED VGG16

cnn=modelo()

cnn.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(lr=lr),
            metrics=['accuracy'])


histoty=cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 100 steps
Epoch 1/100
  3/100 [..............................] - ETA: 12:27 - loss: 2.3797 - accuracy: 0.1500

KeyboardInterrupt: 

In [ ]:
histoty

In [ ]:
!pip install -U matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.figure(0)  
plt.plot(histoty.history['acc'],'r')  
plt.plot(histoty.history['val_acc'],'g')  
plt.xticks(np.arange(0, 21, 2.0))  
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Accuracy")  
plt.title("Training Accuracy vs Validation Accuracy")  
plt.legend(['train','validation'])

plt.figure(1)  
plt.plot(histoty.history['loss'],'r')  
plt.plot(histoty.history['val_loss'],'g')  
plt.xticks(np.arange(0, 21, 2.0))  
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Loss")  
plt.title("Training Loss vs Validation Loss")  
plt.legend(['train','validation'])

plt.show()  

In [ ]:

import keras
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model
import tensorflow as tf 
longitud, altura = 224, 224#tamaño de la imagen
modelo = './modelo/modelo.h5'#direccion del modelo
pesos_modelo = './modelo/pesos.h5'#direccion de los pesos
cnn = tf.keras.models.load_model(modelo)#cargar modelo
cnn.load_weights(pesos_modelo)#cargar pesos
#funcion de prediccion
def predict(file):
  x = tf.keras.preprocessing.image.load_img(file, target_size=(longitud, altura))#cargar la imagen
  x = tf.keras.preprocessing.image.img_to_array(x)#tansformar imagen a arreglo
  x = np.expand_dims(x, axis=0)#en el eje 0 se agrega una dimención extra para procesar la información sin problema
  array = cnn.predict(x)#se llama a la red para realizar la predicción
  result = array[0]#obtenemos el resultado
  answer = np.argmax(result)#nos entrega el indice del valor mas alto
  #clasificación del resultado
  if answer == 0:
    print(file+" pred: 0")
  elif answer == 1:
    print(file+" pred: 1")
  elif answer == 2:
    print(file+" pred: 2")
  elif answer == 3:
    print(file+" pred: 3")
  elif answer == 4:
    print(file+" pred: 4")
  elif answer == 5:
    print(file+" pred: 5")
  elif answer == 6:
    print(file+" pred: 6")
  elif answer == 7:
    print(file+" pred: 7")
  elif answer == 8:
    print(file+" pred: 8")
  elif answer == 9:
    print(file+" pred: 9")
  elif answer == 10:
    print(file+" pred: 10")
  return answer

In [0]:
from google.colab import files

In [0]:
from os import scandir, getcwd
def ls(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [0]:
lista_arq = ls('./data/validacion/pan')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/pan/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/lacteo')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/lacteo/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))


In [0]:
lista_arq = ls('./data/validacion/postre')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/postre/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/huevos')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/huevos/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/fritura')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/fritura/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/carne')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/carne/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/pasta')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/pasta/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/arroz')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/arroz/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/marisco')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/marisco/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/sopa')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/sopa/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:
lista_arq = ls('./data/validacion/frutaverdura')   # no especificar ruta para tomar el directorio actual
cont0=0
cont1=0
cont2=0
cont3=0
cont4=0
cont5=0
cont6=0
cont7=0
cont8=0
cont9=0
cont10=0
for i in lista_arq:
   p=predict('./data/validacion/frutaverdura/'+i)
   if(p==0):
     cont0=cont0+1
   if(p==1):
     cont1=cont1+1
   if(p==2):
     cont2=cont2+1
   if(p==3):
     cont3=cont3+1
   if(p==4):
     cont4=cont4+1
   if(p==5):
     cont5=cont5+1
   if(p==6):
     cont6=cont6+1
   if(p==7):
     cont7=cont7+1
   if(p==8):
     cont8=cont8+1
   if(p==9):
     cont9=cont9+1
   if(p==10):
     cont10=cont10+1
print(100*cont0/len(lista_arq))
print(100*cont1/len(lista_arq))
print(100*cont2/len(lista_arq))
print(100*cont3/len(lista_arq))
print(100*cont4/len(lista_arq))
print(100*cont5/len(lista_arq))
print(100*cont6/len(lista_arq))
print(100*cont7/len(lista_arq))
print(100*cont8/len(lista_arq))
print(100*cont9/len(lista_arq))
print(100*cont10/len(lista_arq))

In [0]:

files.download('./modelo/modelo.h5')
files.download('./modelo/pesos.h5')

In [0]:
lista_arq = ls('./evaluation')   # no especificar ruta para tomar el directorio actual
true=0

for i in lista_arq:
   p=predict('./evaluation/'+i)
   if(str(p)==i[0]):
     true=true+1
print(100*true/len(lista_arq))